In [13]:
# %env PATH_FILE_PRIVATE_KEY_ZEROMQ_CLIENT=/home/adam/Engineering/Docker/smart_broker/private_keys/client.key_secret
# %env  PATH_FILE_PRIVATE_KEY_ZEROMQ_SERVER=/home/adam/Engineering/Docker/smart_broker/private_keys/server.key_secret
# %env PATH_DIR_PUBLIC_KEY_ZEROMQ=/home/adam/Engineering/Docker/smart_broker/public_keys
# %env PATH_DIR_DATASET=/home/adam/datascience/data

env: PATH_FILE_PRIVATE_KEY_ZEROMQ_CLIENT=/home/adam/Engineering/Docker/smart_broker/private_keys/client.key_secret
env: PATH_DIR_PUBLIC_KEY_ZEROMQ=/home/adam/Engineering/Docker/smart_broker/public_keys


# Connect to the SCN

# Data Manipulation
## 1. Bring Data to dataframe state

In [19]:
import requests

def create_data_model_date_frame(data_frame_name):
    payload = {"data_frame_name": data_frame_name}
    result = requests.post("http://127.0.0.1:8000/data_model/new_data_frame_model", params=payload)
    return result.json()["data_model_id"]

def data_model_add_series(data_model_id, series_name, measurement_source_name):
    payload = {"data_model_id": data_model_id,
                "series_name": series_name,
                "measurement_source_name": measurement_source_name}
    result = requests.post("http://127.0.0.1:8000/data_model/data_frame/"+data_model_id+"/add_new_series_model", params=payload)
    return result.json()["data_model_id"]

def get_data_model_tabular():
    result = requests.post("http://127.0.0.1:8000/data_model/new_tabular_model", params=None)
    return result.json()["data_frame_tabular_id"]

def tabular_model_add_dataframe_model(data_model_dataframe_id, data_model_tabular_id):
    payload = {"data_model_dataframe_id": data_model_dataframe_id,
                "data_model_tabular_id": data_model_tabular_id}
    result = requests.post("http://127.0.0.1:8000/data_model/tabular/"+data_model_tabular_id+"/add_dataframe", params=payload)
    return result.json()["data_frame_tabular_id"]

def dataset_tabular_fhirv1(dataset_federation_id, dataset_federation_name, data_model_tabular_id):
    payload = {"dataset_federation_id": dataset_federation_id,
                "dataset_federation_name": dataset_federation_name,
                "data_model_tabular_id": data_model_tabular_id}
    result = requests.post("http://127.0.0.1:8000/ingestion/dataset_tabular/fhirv1", params=payload)
    return result.json()["dataset_id"]

def data_frame_tabular_select_dataframe_model(data_frame_tabular_id, data_frame_name):
    payload = {"data_frame_tabular_id": data_frame_tabular_id,
                "data_frame_name": data_frame_name}
    result = requests.post("http://127.0.0.1:8000/data_frame_tabular/select_dataframe/"+tabular_dataset_id, params=payload)
    return result.json()["data_frame_id"]
    
def data_frame_select_series(data_frame_id, series_name):
    payload = {"data_frame_id": data_frame_id,
                "series_name": series_name}
    result = requests.post("http://127.0.0.1:8000/data_frame/select_series/"+data_frame_id, params=payload)
    return result.json()["series_id"]


In [20]:
import requests

data_model_id = create_data_model_date_frame("data_frame_0")
data_model_add_series(data_model_id, "bmi_mean", "Observation:Body Mass Index")
data_model_add_series(data_model_id, "bmi_first", "Observation:Body Mass Index")
data_model_add_series(data_model_id, "bmi_last", "Observation:Body Mass Index")
data_model_tabular_id = get_data_model_tabular()
data_model_tabular_id = tabular_model_add_dataframe_model(data_model_id, data_model_tabular_id)

dataset_federation_id = "a892f738-4f6f-11ed-bdc3-0242ac120002"
dataset_federation_name = "r4sep2019_csvv1_20_1"
tabular_dataset_id = dataset_tabular_fhirv1(dataset_federation_id, dataset_federation_name, data_model_tabular_id)
data_frame_id = data_frame_tabular_select_dataframe_model(tabular_dataset_id, "data_frame_0")
series_1_id = data_frame_select_series(data_frame_id, "bmi_mean")
series_2_id = data_frame_select_series(data_frame_id, "bmi_last")

print(series_1_id+" , "+series_2_id)

8ca48ba8-42d4-4596-89c8-8d864edde0e6 , fc376bf4-9701-4bab-89ef-bf0bfe466a6e


In [22]:
def count(series_id):
    payload = {"series_id": series_id}
    result = requests.post("http://127.0.0.1:8000/statistics/count/"+series_id, params=payload)
    return result.json()["count"]

def mean(series_id):
    payload = {"series_id": series_id}
    result = requests.post("http://127.0.0.1:8000/statistics/mean/"+series_id, params=payload)
    return result.json()["mean"]

def chisquare(series_1_id, series_2_id):
    payload = {"series_1_id": series_1_id,
                "series_2_id": series_2_id}
    result = requests.post("http://127.0.0.1:8000/statistics/chisquare/"+series_1_id+"/"+series_2_id, params=payload)
    return result.json()["chisquare"]

def kolmogorovSmirnovTest(series_1_id, type_distribution, type_ranking):
    payload = {"series_1_id": series_1_id,
                "type_distribution": type_distribution,
                "type_ranking": type_ranking}
    result = requests.post("http://127.0.0.1:8000/statistics/kolmogorovSmirnovTest/"+series_1_id, params=payload)
    return result.json()["kolmogorov_smirnov_test"]

def kurtosis(series_id):
    payload = {"series_id": series_id}
    result = requests.post("http://127.0.0.1:8000/statistics/kurtosis/"+series_id, params=payload)
    return result.json()["kurtosis"]

def levene_test(series_1_id, series_2_id):
    payload = {"series_1_id": series_1_id,
                "series_2_id": series_2_id}
    result = requests.post("http://127.0.0.1:8000/statistics/levene_test/"+series_1_id+"/"+series_2_id, params=payload)
    return result.json()['f_statistic_sail'], result.json()['p_value_sail']

def mann_whitney_u_test(series_1_id, series_2_id, alternative, type_ranking):
    payload = { "series_1_id": series_1_id,
                "series_2_id": series_2_id,
                "alternative": alternative,
                "type_ranking": type_ranking}
    result = requests.post("http://127.0.0.1:8000/statistics/mann_whitney_u_test/"+series_1_id+"/"+series_2_id, params=payload)
    return result.json()['w_statistic_sail'], result.json()['p_value_sail']

def min_max(series_1_id):
    payload = {"series_id": series_1_id}
    result = requests.post("http://127.0.0.1:8000/statistics/min_max/"+series_1_id, params=payload)
    return result.json()['min_sail'], result.json()['max_sail']

def paired_t_test(series_1_id, series_2_id, alternative):
    payload = {"series_1_id": series_1_id,
                "series_2_id": series_2_id,
                "alternative": alternative}
    result = requests.post("http://127.0.0.1:8000/statistics/paired_t_test/"+series_1_id+"/"+series_2_id, params=payload)
    return result.json()['t_statistic_sail'], result.json()['p_value_sail']

def pearson(series_1_id, series_2_id, alternative):
    payload = {"series_1_id": series_1_id,
                "series_2_id": series_2_id,
                "alternative": alternative}
    result = requests.post("http://127.0.0.1:8000/statistics/pearson/"+series_1_id+"/"+series_2_id, params=payload)
    return result.json()['pearson_sail'], result.json()['p_value_sail']

def skewness(series_id):
    payload = {"series_id": series_id}
    result = requests.post("http://127.0.0.1:8000/statistics/skewness/"+series_id, params=payload)
    return result.json()["skewness_sail"]

def spearman(series_1_id, series_2_id, alternative, type_ranking):
    payload = { "series_1_id": series_1_id,
                "series_2_id": series_2_id,
                "alternative": alternative,
                "type_ranking": type_ranking}
    result = requests.post("http://127.0.0.1:8000/statistics/spearman/"+series_1_id+"/"+series_2_id, params=payload)
    return result.json()['spearman_sail'], result.json()['p_value_sail']

def student_t_test(series_1_id, series_2_id, alternative):
    payload = {"series_1_id": series_1_id,
                "series_2_id": series_2_id,
                "alternative": alternative}
    result = requests.post("http://127.0.0.1:8000/statistics/student_t_test/"+series_1_id+"/"+series_2_id, params=payload)
    return result.json()['t_statistic_sail'], result.json()['p_value_sail']

def variance(series_id):
    payload = {"series_id": series_id}
    result = requests.post("http://127.0.0.1:8000/statistics/variance/"+series_id, params=payload)
    return result.json()["variance_sail"]

def welch_t_test(series_1_id, series_2_id, alternative):
    payload = {"series_1_id": series_1_id,
                "series_2_id": series_2_id,
                "alternative": alternative}
    result = requests.post("http://127.0.0.1:8000/statistics/welch_t_test/"+series_1_id+"/"+series_2_id, params=payload)
    return result.json()['t_statistic_sail'], result.json()['p_value_sail']

def wilcoxon_signed_rank_test(series_1_id, series_2_id, alternative, type_ranking):
    payload = { "series_1_id": series_1_id,
                "series_2_id": series_2_id,
                "alternative": alternative,
                "type_ranking": type_ranking}
    result = requests.post("http://127.0.0.1:8000/statistics/wilcoxon_signed_rank_test/"+series_1_id+"/"+series_2_id, params=payload)
    return result.json()['w_statistic_sail'], result.json()['p_value_sail']


type_distribution="normalunit" 
type_ranking="unsafe"
alternative = "two-sided"
print(count(series_1_id))
print(mean(series_1_id))
print(chisquare(series_1_id, series_2_id))
# print(kolmogorovSmirnovTest(series_1_id, type_distribution, type_ranking)) # RANK DATAFRAME NOT FOUND
print(kurtosis(series_1_id))
# # print(levene_test(series_1_id, series_2_id)) # NDARRAY SERIALISATION
# # mann_whitney_u_test(series_1_id, series_2_id, alternative, type_ranking)  # RANK DATAFRAME NOT FOUND
print(min_max(series_1_id))
# print(paired_t_test(series_1_id, series_2_id, alternative)) # ValueError: Out of range float values are not JSON compliant
# print(pearson(series_1_id, series_2_id, alternative)) # ValueError: math domain error
print(skewness(series_1_id))
# print(spearman(series_1_id, series_2_id, alternative, type_ranking))  # RANK DATAFRAME NOT FOUND
print(student_t_test(series_1_id, series_2_id, alternative))
print(variance(series_1_id))
print(welch_t_test(series_1_id, series_2_id, alternative))
# print(wilcoxon_signed_rank_test(series_1_id, series_2_id, alternative, type_ranking)) # RANK DataFrame not loaded: 30459bd2-2880-4829-8d05-1e6bb9c1a71f







20
22.76433814224586
[380.0, 0.23583148842099294]
-1.225866948308427
(11.76951154694196, 30.172100539128525)
-0.08041628194553505
(0.0, 0.25)
28.617281263722433
(0.0, 0.25)
